<h1><span style = "color:red">MySQL File Upload</span></h1>
<div class = "alert alert-block alert-info" style = "margin-top:20-px">

This script loads two xlsx file where information on will used extract information from another file and concated to produce different files. Based on certain conditions from the uploaded files, certain information is queried from the database to validate the information. If certain conditions are meet, say for example new information, that information is loaded into the database for future reference and validation.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None
import mysql.connector


df = pd.read_excel("Type_of_upload.xlsx")
Upl = pd.read_excel("Upload.xlsx", dtype=object)

ASN = Upl[Upl['Code']=="ASN"]
Upl = Upl[Upl['Code']!="ASN"]

def check(ASN,Upl):   
    if not list(ASN['Code']):
        return Upl
    else:
        connection = mysql.connector.connect(host='localhost', user='root',password = '1234@1', database = 'flonet')
        my_data = pd.read_sql("SELECT productcode FROM pfep",connection)
        connection.close()
        No_PFEP = ASN[~ASN['ProductCode'].isin(my_data['productcode'])]
        Upl = pd.concat([Upl, No_PFEP])
        
    return Upl
Upl = check(ASN,Upl)
Upl.reset_index(inplace = True, drop = True)

#PFEP Setting
#PFEP Setting
c = []
for i in Upl['Code']:
    a = df[df['Code'] ==i]
    c.append(a)
PFEP = pd.concat(c)
PFEP.reset_index(drop=True,inplace = True)

PFEP['ProductCode'].fillna(Upl['ProductCode'],inplace = True)

PFEP.drop(columns =['Code'], inplace = True)
PFEP['ProductCode']= PFEP['ProductCode'].astype(object)

#StorageUnitType Setting
StorageUnitType = PFEP[['ProductCode','PutAwaySUT']]
StorageUnitType.set_axis(['ProductCode', 'StorageUnitTypeCode'], axis=1, inplace=True)

#MovementType
MovementType =PFEP[['ProductCode']]
for i in MovementType:
    MovementType['MovementCode'] = "A"

Update = PFEP[PFEP['ProductCode'].isin(ASN['ProductCode'])]

def update_pfep(Update):
    if not list(Update['ProductCode']):
        print("No Database Update")
    else:
        Update = Update.astype(object).where(pd.notnull(Update), None)
        connection = mysql.connector.connect(host='localhost', user='root',password = '1234@1', database = 'flonet')
        cursor = connection.cursor()
        cols = "`,`".join([str(i) for i in Update.columns.tolist()])
        for i,row in Update.iterrows():
            sql = "INSERT INTO `pfep` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))
            connection.commit()
        connection.close()
    return Update
update_pfep(Update)

print("Done")